In [7]:
import pandas as pd

# Rutas de los archivos
sales_file_path = './sales_test.csv'
products_file_path = './products.csv'
stock_file_path = './initial_stock.csv'

# Cargar los datasets
sales = pd.read_csv(sales_file_path)
products = pd.read_csv(products_file_path)
stock = pd.read_csv(stock_file_path)

## Classic Methods MENSUAL RESTOCK


In [47]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [86]:
# Renombrar la columna 'id' a 'product_id' en products
products.rename(columns={'id': 'product_id'}, inplace=True)

# Combinar los datos
data = pd.merge(sales, products, on='product_id', how='left')
data = pd.merge(data, stock, on='product_id', how='left')

# Convertir la columna 'date' a datetime si no lo es
data['date'] = pd.to_datetime(data['date'])

# Crear características adicionales
data['Days to Expire'] = (pd.to_datetime(data['batch_expiry_date']) - data['date']).dt.days
data['Total Revenue'] = data['total_price']
data['Price_USD'] = data['Price_USD']
data['Remaining stock'] = data['remaining_stock']

data["month"] = data["date"].dt.month
data["year"] = data["date"].dt.year


monthly_data = data.groupby(["month", "year", "product_id"]).agg(
    total_revenue=('Total Revenue', 'sum'),
    average_price=('Price_USD', 'mean'),
    stock=('remaining_stock', 'last'),
    average_days_to_expire=('Days to Expire', 'mean'),
    quantity = ('quantity_x', 'sum')
).reset_index()


# Seleccionar las características y el target para el modelo mensual
features_monthly = monthly_data[["month", "year",'product_id','total_revenue', 'average_price', 'stock',"average_days_to_expire"]]
target_monthly = monthly_data['quantity']

# Dividir los datos mensuales en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    features_monthly, target_monthly, test_size=0.2, random_state=42
)
# Escalar las características
scaler_monthly = StandardScaler()
X_train_scaled = scaler_monthly.fit_transform(X_train)
X_test_scaled = scaler_monthly.transform(X_test)

print(features_monthly.index)

X_train_scaled = np.array(X_train_scaled)
X_test_scaled = np.array(X_test_scaled)
y_train = np.array(y_train).flatten()  
y_test = np.array(y_test).flatten()

RangeIndex(start=0, stop=1496, step=1)


In [79]:
# Evaluar múltiples modelos
results = {}

# Random Forest Regressor
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train_scaled, y_train)
y_pred_rf = rf_model.predict(X_test_scaled)
results['Random Forest'] = {
    'MSE': mean_squared_error(y_test, y_pred_rf),
    'R2': r2_score(y_test, y_pred_rf)
}

# Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train_scaled, y_train)
y_pred_linear = linear_model.predict(X_test_scaled)
results['Linear Regression'] = {
    'MSE': mean_squared_error(y_test, y_pred_linear),
    'R2': r2_score(y_test, y_pred_linear)
}

# Polynomial Regression
degrees = range(1, 6)  # Testing degrees from 1 to 5
poly_results = {}

for degree in degrees:
    poly = PolynomialFeatures(degree=degree)
    X_train_poly = poly.fit_transform(X_train_scaled)
    X_test_poly = poly.transform(X_test_scaled)

    poly_model = LinearRegression()
    poly_model.fit(X_train_poly, y_train)
    y_pred_poly = poly_model.predict(X_test_poly)

    poly_results[degree] = {
        'MSE': mean_squared_error(y_test, y_pred_poly),
        'R2': r2_score(y_test, y_pred_poly)
    }

# Find the best degree and add to results
best_degree = max(poly_results, key=lambda d: poly_results[d]['R2'])
results['Polynomial Regression (Best Degree)'] = {
    'Degree': best_degree,
    'MSE': poly_results[best_degree]['MSE'],
    'R2': poly_results[best_degree]['R2']
}

# Gradient Boosting Regressor
gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train_scaled, y_train)
y_pred_gb = gb_model.predict(X_test_scaled)
results['Gradient Boosting'] = {
    'MSE': mean_squared_error(y_test, y_pred_gb),
    'R2': r2_score(y_test, y_pred_gb)
}

# K-Nearest Neighbors Regressor
knn_model = KNeighborsRegressor()
knn_model.fit(X_train_scaled, y_train)
y_pred_knn = knn_model.predict(X_test_scaled)
results['KNN Regressor'] = {
    'MSE': mean_squared_error(y_test, y_pred_knn),
    'R2': r2_score(y_test, y_pred_knn)
}

# Support Vector Regressor
svr_model = SVR()
svr_model.fit(X_train_scaled, y_train)
y_pred_svr = svr_model.predict(X_test_scaled)
results['Support Vector Regressor'] = {
    'MSE': mean_squared_error(y_test, y_pred_svr),
    'R2': r2_score(y_test, y_pred_svr)
}

# Neural Network
nn_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='linear')
])

nn_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
nn_model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test), epochs=50, batch_size=32, verbose=0)
nn_loss, nn_mae = nn_model.evaluate(X_test_scaled, y_test, verbose=0)
y_pred_nn = nn_model.predict(X_test_scaled).flatten()
results['Neural Network'] = {
    'MSE': mean_squared_error(y_test, y_pred_nn),
    'R2': r2_score(y_test, y_pred_nn),
    'MAE': nn_mae
}

# Output results
print('Model Performance Summary:')
for model, metrics in results.items():
    if 'Degree' in metrics:
        print(f"{model} (Degree {metrics['Degree']}): MSE={metrics['MSE']}, R2={metrics['R2']}")
    else:
        print(f"{model}: MSE={metrics['MSE']}, R2={metrics['R2']}")


/home/rebooter/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
Model Performance Summary:
Random Forest: MSE=4.976864666666666, R2=0.988731723269077
Linear Regression: MSE=45.42552862722214, R2=0.8971506236349333
Polynomial Regression (Best Degree) (Degree 3): MSE=0.32170226619702946, R2=0.99927162372231
Gradient Boosting: MSE=2.6408528459520304, R2=0.9940207615302178
KNN Regressor: MSE=27.035333333333334, R2=0.9387884465589086
Support Vector Regressor: MSE=51.97160474457883, R2=0.882329445617757
Neural Network: MSE=9.502740578060736, R2=0.9784845444456344


### Predicción Enero y Febrero 2025

##### De esta manera sabemos cuanto stock pedir en febrero de 2025 e incluimos un colchón de seguridad

In [81]:
features_monthly.head()

,month,year,product_id,total_revenue,average_price,stock,average_days_to_expire
0,1,2020,1,8054.64,149.16,23,1064.880000
1,1,2020,2,3736.80,62.28,20,891.888889
2,1,2020,3,3099.00,51.65,24,1028.296296
3,1,2020,4,4417.50,71.25,37,1098.785714
4,1,2020,5,4701.60,78.36,33,1053.964286


In [106]:
january_input = pd.DataFrame(
    {
        'month': [1],  
        'year': [2025],
        'product_id': [1],
        'total_revenue': [3099.00],
        'average_price': [149.16], 
        'stock': [20],
        'average_days_to_expire': [891.888889],
    }
)

january_input = scaler_monthly.transform(january_input)
print(january_input)

[[-1.59296748  2.13347128 -1.67051796  0.69266137  1.8535728   0.71420461
   5.26284478]]


In [112]:
january_prediction = gb_model.predict(january_input)
print(january_prediction)
january_prediction = int(january_prediction.round())
print(january_prediction)
print(type(january_prediction))

[28.64908622]
29
<class 'int'>


/tmp/ipykernel_10787/3468170727.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  january_prediction = int(january_prediction.round())


In [151]:
february_input = pd.DataFrame(
    {
        'month': [2],  
        'year': [2025],
        'product_id': [1],
        'total_revenue': [3080.00],
        'average_price': [145.16], 
        'stock': [50-january_prediction],
        'average_days_to_expire': [880.888889],
    }
)

product_id_and_prediction = []


february_prediction = gb_model.predict(february_input)
february_prediction = int(february_prediction.round())


for idx, row in february_input.iterrows():
    product_data = {
        'id': int(row['product_id']),         
        'quantity': int(february_prediction*1.2)    
    }
    
    product_id_and_prediction.append(product_data)

print(product_id_and_prediction)


[{'id': 1, 'quantity': 61}]


/home/rebooter/anaconda3/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/tmp/ipykernel_10787/2974192523.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  february_prediction = int(february_prediction.round())


##### Stock a comprar en febrero

In [153]:
product_name_and_id = dict(zip(products["product_id"], products["Product Name"]))
print(product_name_and_id)

{1: 'Product 21', 2: 'Product 22', 3: 'Product 23', 4: 'Product 24', 5: 'Product 25', 6: 'Product 26', 7: 'Product 27', 8: 'Product 28', 9: 'Product 29', 10: 'Product 30', 11: 'Product 31', 12: 'Product 32', 13: 'Product 33', 14: 'Product 34', 15: 'Product 35', 16: 'Product 36', 17: 'Product 37', 18: 'Product 38', 19: 'Product 39', 20: 'Product 40', 21: 'Product 41', 22: 'Product 42', 23: 'Product 43', 24: 'Product 44', 25: 'Product 45', 26: 'Product 46', 27: 'Product 47', 28: 'Product 48', 29: 'Product 49', 30: 'Product 50'}


In [155]:
product_diccionario = {}

for key, value in product_name_and_id.items():
    for product in product_id_and_prediction:
        if product['id'] == key:  
            product_diccionario[value] = product['quantity']  

print(product_diccionario)

{'Product 21': 61}


In [157]:
import json

product_json = json.dumps(product_diccionario, indent=4, ensure_ascii=False)

print(product_json)

{
    "Product 21": 61
}


#### Prophet --- Mensual restocking

In [163]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from datetime import datetime

# Primero preparamos los datos en el formato que requiere Prophet
def prepare_prophet_data(monthly_data):
    # Prophet requiere columnas 'ds' (fecha) y 'y' (target)
    prophet_data = monthly_data.copy()
    
    # Crear fechas combinando mes y año
    prophet_data['ds'] = prophet_data.apply(
        lambda x: datetime(int(x['year']), int(x['month']), 1), axis=1
    )
    
    # Renombrar quantity como 'y'
    prophet_data['y'] = prophet_data['quantity']
    
    # Agregar las características adicionales como regressors
    prophet_data['total_revenue_scaled'] = (prophet_data['total_revenue'] - 
                                          prophet_data['total_revenue'].mean()) / prophet_data['total_revenue'].std()
    prophet_data['average_price_scaled'] = (prophet_data['average_price'] - 
                                          prophet_data['average_price'].mean()) / prophet_data['average_price'].std()
    prophet_data['stock_scaled'] = (prophet_data['stock'] - 
                                   prophet_data['stock'].mean()) / prophet_data['stock'].std()
    prophet_data['days_to_expire_scaled'] = (prophet_data['average_days_to_expire'] - 
                                            prophet_data['average_days_to_expire'].mean()) / prophet_data['average_days_to_expire'].std()
    
    return prophet_data

def train_evaluate_prophet(data, product_id=None):
    """
    Entrena y evalúa un modelo Prophet para un producto específico o todos los productos
    """
    if product_id is not None:
        data = data[data['product_id'] == product_id]
    
    # Preparar datos para Prophet
    prophet_data = prepare_prophet_data(data)
    
    # División train-test preservando el orden temporal
    train_size = int(len(prophet_data) * 0.8)
    train_data = prophet_data.iloc[:train_size]
    test_data = prophet_data.iloc[train_size:]
    
    # Inicializar y configurar el modelo Prophet
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False,
        seasonality_mode='multiplicative'
    )
    
    # Agregar regressors
    model.add_regressor('total_revenue_scaled')
    model.add_regressor('average_price_scaled')
    model.add_regressor('stock_scaled')
    model.add_regressor('days_to_expire_scaled')
    
    # Entrenar el modelo
    model.fit(train_data)
    
    # Preparar datos futuros para predicción
    future = model.make_future_dataframe(periods=len(test_data), freq='M')
    future = future.merge(test_data[['ds', 'total_revenue_scaled', 'average_price_scaled', 
                                   'stock_scaled', 'days_to_expire_scaled']], 
                         on='ds', how='left')
    
    # Realizar predicciones
    forecast = model.predict(future)
    
    # Evaluar el modelo
    y_true = test_data['y'].values
    y_pred = forecast.tail(len(test_data))['yhat'].values
    
    results = {
        'Prophet': {
            'MSE': mean_squared_error(y_true, y_pred),
            'R2': r2_score(y_true, y_pred)
        }
    }
    
    return model, forecast, results

# Ejemplo de uso
def run_prophet_analysis(monthly_data):
    # Análisis global
    print("Análisis global de todos los productos:")
    model, forecast, results = train_evaluate_prophet(monthly_data)
    print(f"Prophet Global: MSE={results['Prophet']['MSE']}, R2={results['Prophet']['R2']}")
    
    # Análisis por producto
    unique_products = monthly_data['product_id'].unique()
    product_results = {}
    
    for product in unique_products:
        print(f"\nAnálisis para producto {product}:")
        _, _, prod_results = train_evaluate_prophet(monthly_data, product)
        product_results[product] = prod_results['Prophet']
        print(f"Prophet Producto {product}: MSE={prod_results['Prophet']['MSE']}, R2={prod_results['Prophet']['R2']}")
    
    return model, forecast, results, product_results

# Para ejecutar el análisis:
# model, forecast, results, product_results = run_prophet_analysis(monthly_data)

ModuleNotFoundError: No module named 'prophet'